In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os

# ==============================================================================
# PROJECT PARAMETERS (Confirmed from Data/Metadata/USGS)
# ==============================================================================

# --- 1. KOOTENAI RIVER PARAMETERS (Large River: Low Self-Cleansing Baseline) ---
K_Q = 198.40  # Mean Discharge (m^3/s)
K_M_INJ_KG = 14.514  # Injected Mass (kg)
K_L_M = 14659.0  # Distance to Sonde 1A (m)
K_T_INJ = '2017-09-27 04:10:40'  # Injection End Time (UTC)
K_STATION = '1A'

# --- 2. BIG PINEY RIVER PARAMETERS (Small River: High Self-Cleansing) ---
BP_Q = 5.777  # Mean Discharge (m^3/s) - CONFIRMED from USGS NWIS data
BP_M_INJ_KG = 2.40  # Injected Mass (kg)
BP_L_M = 1500.0  # Distance to Transect 1 (m)
BP_T_INJ = '2021-08-25 20:07:00'  # Injection Time (UTC)
BP_STATION = 1

# ==============================================================================
# CORE CALCULATION FUNCTION: Moment Method for Initial Assimilation Factor (Ka)
# ==============================================================================

def calculate_initial_ka(df, Q, M_INJ_KG, T_INJ_STR, L_M, river_name, time_col, conc_col, station_identifier, station_value):
    """Calculates the Initial Assimilation Factor (Ka) for a river reach and returns data for plotting, including the max peak."""
    df = df.copy()
    M_INJ_UG = M_INJ_KG * 1e9 # Convert kg to micrograms (ug)

    # 1. Time Normalization
    df[time_col] = pd.to_datetime(df[time_col])
    T_INJ = pd.to_datetime(T_INJ_STR)
    df['Time_s'] = (df[time_col] - T_INJ).dt.total_seconds()
    
    # 2. Filter Data 
    df_station = df[df[station_identifier] == station_value].copy()
    df_filtered = df_station[df_station['Time_s'] >= 0].sort_values(by='Time_s')
    
    times = df_filtered['Time_s'].values
    concentrations = df_filtered[conc_col].values # Conc units are ug/L or ppb (equivalent)
    
    # --- NEW: Calculate Peak Concentration ---
    peak_concentration = np.max(concentrations) if len(concentrations) > 0 else 0

    # 3. Moment Calculation (M0 and t_bar)
    if len(times) < 5 or M_INJ_UG <= 0 or np.sum(concentrations) == 0:
        return {'River': river_name, 'Q (m^3/s)': Q, 'L (km)': L_M / 1000, 'M_Injected (kg)': M_INJ_KG, 'M_Passed (kg)': 0, 'M_Loss (%)': 100, 'Ka_initial (1/hr)': np.nan, 'Plot_Data': df_filtered, 'Peak_Concentration (ug/L)': peak_concentration}

    M0 = np.trapezoid(concentrations, times) 
    M1 = np.trapezoid(concentrations * times, times)
    t_bar = M1 / M0 
    
    M_PASSED_UG = M0 * Q * 1000 

    # 4. Assimilation Factor (Ka) Calculation
    Ka_s = - (1 / t_bar) * np.log(M_PASSED_UG / M_INJ_UG)
    Ka_hr = Ka_s * 3600 # Convert to per hour

    return {
        'River': river_name,
        'Q (m^3/s)': Q,
        'L (km)': L_M / 1000,
        'M_Injected (kg)': M_INJ_KG,
        'M_Passed (kg)': M_PASSED_UG / 1e9,
        'M_Loss (%)': 100 * (1 - (M_PASSED_UG / M_INJ_UG)),
        'Ka_initial (1/hr)': Ka_hr,
        'Plot_Data': df_filtered,
        'Peak_Concentration (ug/L)': peak_concentration  # Added to the output dictionary
    }

def plot_btcs(results):
    """Generates and saves a separate BTC plot for Kootenai and Big Piney."""
    for result in results:
        river_name = result['River']
        Q = result['Q (m^3/s)']
        df_plot = result['Plot_Data'].copy()
        
        # Determine the concentration column name dynamically
        conc_col = 'DyeConc' if river_name == 'Kootenai' else 'rwt_ppb'
        if conc_col not in df_plot.columns:
            conc_col = 'DyeConc_ugL' if 'DyeConc_ugL' in df_plot.columns else conc_col

        plt.figure(figsize=(10, 6))
        
        # Plot Time in HOURS
        plt.plot(
            df_plot['Time_s'] / 3600, 
            df_plot[conc_col], 
            label=f"Q={Q:.1f} m³/s",
            linewidth=2,
            alpha=0.8,
            color='teal' if river_name == 'Kootenai' else 'firebrick',
            marker='.',
            markersize=3
        )

        plt.title(f'{river_name} River Breakthrough Curve (BTC) at First Station')
        plt.xlabel('Time Post-Injection (hours)')
        plt.ylabel('Dye Concentration (µg/L or ppb)')
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.legend(title="Flow Condition", loc='upper right')
        plt.tight_layout()
        
        # Save the plot image
        plot_path = f'{river_name}_BTC_Plot.png'
        plt.savefig(plot_path)
        plt.close() # Close plot figure to prevent overlap
        print(f"\n✅ BTC plot saved to {plot_path}")

# ==============================================================================
# EXECUTION AND EXPORT
# ==============================================================================

# --- 1. KOOTENAI RIVER EXECUTION ---
df_kootenai = pd.read_csv('KootenaiMooredSondeConcentrationData.csv')
kootenai_result = calculate_initial_ka(
    df_kootenai, K_Q, K_M_INJ_KG, K_T_INJ, K_L_M, 'Kootenai', 
    'UTCtimeStamp', 'DyeConc', 'Sonde', K_STATION
)

# --- 2. BIG PINEY RIVER EXECUTION ---
df_bp = pd.read_csv('BigPiney_RWTFluorometers.csv')
# Must handle the dual time/date columns for Big Piney
df_bp['UTCtimeStamp'] = pd.to_datetime(df_bp['date'] + ' ' + df_bp['time'])
bp_result = calculate_initial_ka(
    df_bp, BP_Q, BP_M_INJ_KG, BP_T_INJ, BP_L_M, 'Big Piney', 
    'UTCtimeStamp', 'rwt_ppb', 'transect_ID', BP_STATION
)

# --- 3. CREATE FINAL COMPARISON TABLE ---
df_final_comparison = pd.DataFrame([bp_result, kootenai_result])

# Clean up M_Loss percentage
df_final_comparison['M_Loss (%)'] = df_final_comparison['M_Loss (%)'].apply(lambda x: max(0, x))

# --- 4. GENERATE PLOTS ---
plot_btcs([bp_result, kootenai_result])

# --- 5. EXPORT TO CSV FILE ---
CSV_FILE_PATH = "Ka_Comparison_Results.csv"
# Updated list of columns for final export, including the new 'Peak_Concentration'
df_final_comparison[['River', 'Q (m^3/s)', 'L (km)', 'M_Injected (kg)', 'M_Passed (kg)', 'M_Loss (%)', 'Ka_initial (1/hr)', 'Peak_Concentration (ug/L)']].round(4).to_csv(CSV_FILE_PATH, index=False)


# --- 6. PRINT SUMMARY ---
print("===============================================================")
print("  FINAL COMPARATIVE ASSIMILATION FACTOR (Ka) RESULTS")
print("  BTC plots saved as Big Piney_BTC_Plot.png and Kootenai_BTC_Plot.png")
print("===============================================================")

print("\n--- Summary of Calculated Parameters ---")
# Updated list of columns for the printed summary
print(df_final_comparison[['River', 'Q (m^3/s)', 'L (km)', 'M_Passed (kg)', 'M_Loss (%)', 'Peak_Concentration (ug/L)', 'Ka_initial (1/hr)']].round(4).to_markdown(index=False))


✅ BTC plot saved to Big Piney_BTC_Plot.png

✅ BTC plot saved to Kootenai_BTC_Plot.png
  FINAL COMPARATIVE ASSIMILATION FACTOR (Ka) RESULTS
  BTC plots saved as Big Piney_BTC_Plot.png and Kootenai_BTC_Plot.png

--- Summary of Calculated Parameters ---
| River     |   Q (m^3/s) |   L (km) |   M_Passed (kg) |   M_Loss (%) |   Peak_Concentration (ug/L) |   Ka_initial (1/hr) |
|:----------|------------:|---------:|----------------:|-------------:|----------------------------:|--------------------:|
| Big Piney |       5.777 |    1.5   |          0.8986 |      62.5587 |                       52.87 |              0.0692 |
| Kootenai  |     198.4   |   14.659 |         13.8648 |       4.4731 |                        7.76 |              0.0034 |
